# Setting

## Import

In [1]:
import multiprocessing
import socket
import time

import os
import time
import copy
import random
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import IPython

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# Model

In [10]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock_18(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock_18, self).__init__()
        self.conv_1 = conv3x3(inplanes, planes, stride)
        self.bn_1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv_2 = conv3x3(planes, planes)
        self.bn_2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv_1(x)
        out = self.bn_1(out)
        out = self.relu(out)

        out = self.conv_2(out)
        out = self.bn_2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu(out)

        return out

class ResNet_18(nn.Module):
    def __init__(self, block, layers, num_classes, grayscale):
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3

        super(ResNet_18, self).__init__()
        self.conv1g = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bng = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512*block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes*block.expansion),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1g(x)
        x = self.bng(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x_full = x.view(x.size(0), -1)
        logits = self.fc(x_full)
        #probas = F.softmax(logits, dim=1)
        return logits, x_full

def resnet18(num_classes, GRAYSCALE):
    model = ResNet_18(block=BasicBlock_18, layers=[2,2,2,2], num_classes=num_classes, grayscale=GRAYSCALE)
    return model

# Model Weight Avg

In [ ]:
def running_model_avg(current, next, scale):
    if current == None:
        current = next
        for key in current:
            current[key] = current[key] * scale
    else:
        for key in current:
            current[key] = current[key] + (next[key] * scale)
    return current

In [7]:
def train_client(client_loader, global_model, num_local_epochs, lr):
    local_model = copy.deepcopy(global_model)
    local_model = local_model.to(device)
    local_model.train()
    optimizer = torch.optim.SGD(local_model.parameters(), lr=lr, momentum = 0.9)

    for epoch in range(num_local_epochs):
        for (i, (x,y)) in enumerate(client_loader):
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            local_out, _ = local_model(x)
      
            loss = criterion(local_out, y)
            loss.backward()
            optimizer.step()
    
    
    return local_model

## Communication Setup

In [ ]:
import socket


server_address = ('localhost', 8080) # 항시 Localhost -> Java 가동 서버로 전달 

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    try:
        # 서버에 연결 시도 
        s.connect(server_address)
        
        # 연결 성공 시 서버 정보 출력
        print("정상적으로 서버와 연결이 완료되었습니다.")

            
    except socket.error as e:
        # 연결 실패 시 오류 메시지 출력
        print("통신 도중 오류가 발생했습니다.")
        exit()

# Server Federated Learning

In [16]:
def fed_avg_experiment(global_model, max_round, lr):
    round_accuracy = []
    for t in range(max_round):
        
        print("starting round {}".format(t+1))
        
        global_model.eval()
        global_model = global_model.to('cpu')
        torch.save(global_model, '/model/global_model.pt')
        running_avg = None
        
        #############################################################################################
        
        
        # 모델 전송 부분 (실제로 학습 요청)
        socket.sendall(t) # Round Number 전송 
        
        response = socket.recv(1024)
        if response is not t:
            print("통신 실패")
        else : 
            num_client = response # 연결된 클라이언트 수 
        
        #############################################################################################
        
        # 클라이언트 갯수만큼....
        for k in range(num_client):
            client_model = torch.load(f'/model/client_model_{k+1}')
            running_avg = running_model_avg(running_avg, client_model.state_dict(), 1/num_client)
        


        global_model.load_state_dict(running_avg)
        
        torch.save(global_model, '/model/global_model.pt')
        val_acc = validate(global_model)
        print('Round: {}  Accuracy: {}'.format(t+1, val_acc))
        round_accuracy.append(val_acc)
    return round_accuracy
    

# Validate

In [9]:
def validate(model):
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for (t, (x, y)) in enumerate(test_loader_CIFAR10):
            x = x.to(device)
            y = y.to(device)
            out, _ = model(x)
            correct += torch.sum(torch.argmax(out, dim=1) ==y).item()
            total += x.shape[0]
    return correct/total

In [ ]:
model = resnet18(10, False)
criterion = nn.CrossEntropyLoss()

acc = fed_avg_experiment(global_model=model, max_round=100, lr=0.01)